In [44]:
from functions import util as util
from functions import event_parser as EvtParser
from functions import hit_parser as HitParser

import h5py
import numpy as np
import plotly.graph_objects as go
import yaml

In [45]:
switch_xz = False
# module0, 2x2, 2x2_MR4, ndlar, singlecube
detector = "singlecube"

In [46]:
def draw_cathode_planes(geom_dict, tpc_centers, **kwargs):
    
    pos = np.array(list(geom_dict.values())) / 10.
    traces = []
    for i_mod in range(len(tpc_centers)):
        x = np.array([pos[:, 0].min() + tpc_centers[i_mod][2], pos[:, 0].max() + tpc_centers[i_mod][2]])
        y = np.array([pos[:, 1].min() + tpc_centers[i_mod][1], pos[:, 1].max() + tpc_centers[i_mod][1]])
        z = np.full((2,2),  tpc_centers[i_mod][0])
        
        traces.append(go.Surface(x=x, y=y, z=z, **kwargs))
    
    return traces

In [47]:
def draw_anode_planes(geom_dict, tpc_centers, **kwargs):
    
    pos = np.array(list(geom_dict.values())) / 10.
    traces = []
    
    for i_mod in range(len(tpc_centers)):
        x = np.array([pos[:, 0].min() + tpc_centers[i_mod][2], pos[:, 0].max() + tpc_centers[i_mod][2]])
        y = np.array([pos[:, 1].min() + tpc_centers[i_mod][1], pos[:, 1].max() + tpc_centers[i_mod][1]])
        z1 = np.full((2,2),  pos[:, 2].min() + tpc_centers[i_mod][0])
        z2 = np.full((2,2),  pos[:, 2].max() + tpc_centers[i_mod][0])
        
        traces.append(go.Surface(x=x, y=y, z=z1, **kwargs))
        traces.append(go.Surface(x=x, y=y, z=z2, **kwargs))
    
    return traces

In [48]:
def plot_tracks(tracks, **kwargs):
    
    def to_list(axis):
        return np.column_stack([
            tracks[f'{axis}_start'],
            tracks[f'{axis}_end'],
            np.full(len(tracks), None)
        ]).flatten().tolist()
        
    x, y, z = (to_list(axis) for axis in 'xyz')

    trace = go.Scatter3d(x=z, y=y, z=x, **kwargs)
    
    return trace

In [49]:
f = h5py.File('/Users/emoore06/Work/research/larpix_event_display/data/scmuon_output.h5', 'r')
packets = f['packets'] # readout
segs = f['segments'] # Geant4 truth
assn = f['mc_packets_assn'] # G4-readout association
vtx = f['vertices']
traj = f['trajectories']

In [50]:
'''
    Loading LArPix geometry and run configuration
    Option 1: Use the default, the installed package data
              It's recommended, if you are not doing something unusual.
'''

#run_config, geom_dict = util.detector_configuration(detector)

"\n    Loading LArPix geometry and run configuration\n    Option 1: Use the default, the installed package data\n              It's recommended, if you are not doing something unusual.\n"

In [51]:
'''
    Loading LArPix geometry and run configuration
    Option 2: Point a pre-load LArPix geometry dictionary stored in a pickle file and a run configuration yaml file
'''
# geom_dict = util.load_geom_dict("../src/LarpixParser/config_repo/dict_repo/multi_tile_layout-3.0.40.pkl")

# run_config_path = '../src/LarpixParser/config_repo/ndlar-module.yaml'
# run_config = util.get_run_config(run_config_path)

'\n    Loading LArPix geometry and run configuration\n    Option 2: Point a pre-load LArPix geometry dictionary stored in a pickle file and a run configuration yaml file\n'

In [52]:
'''
    Loading LArPix geometry and run configuration
    Option 3: Build a LArPix geometry dictionary on the fly and load a run configuration yaml file
'''
from functions import geom_to_dict as DictBuilder
geom_repo = '/Users/emoore06/Work/research/larpix_event_display/config_repo'
larpix_layout_name = 'single_tile_layout-2.4.1'
geom_dict = DictBuilder.larpix_layout_to_dict(larpix_layout_name, geom_repo, save_dict=False)

run_config_path = '/Users/emoore06/Work/research/larpix_event_display/config_repo/singlecube.yaml'
run_config = util.get_run_config(run_config_path)

In [53]:
if switch_xz:
    tpc_centers_tp = np.array(run_config['tpc_offsets'])
    tpc_centers = np.copy(tpc_centers_tp)
    tpc_centers[:,0] = tpc_centers_tp[:,2]
    tpc_centers[:,2] = tpc_centers_tp[:,0]
else:
    tpc_centers = run_config['tpc_offsets']

In [54]:
pckt_event_ids = EvtParser.packet_to_eventid(assn, segs)
event_ids = np.unique(pckt_event_ids[pckt_event_ids != -1]) 
t0_grp = EvtParser.get_t0_event(vtx, run_config)

Found no 'beam_duration' in the configuration file


In [55]:
xyzevents = []

for i_ev, evt_id in enumerate(event_ids):

    pckt_mask = pckt_event_ids == evt_id
    packets_ev = packets[pckt_mask]
    assn_ev = assn[pckt_mask]
    seg_ev = segs[segs['event_id'] == evt_id]
    t0 = packets_ev['timestamp'][0]
    
    x,y,z,dQ, v_drift_arr, t_drift_arr = HitParser.hit_parser_charge(t0, packets_ev, geom_dict, run_config, switch_xz)

    x = np.array(x) / 10. #Converting from mm to cm
    y = np.array(y) / 10. #Converting from mm to cm
    z = np.array(z) / 100. #Z value is found using the clock cycle which is 100 ns, so we need to divide out to convert to s here

    xyzevents.append((x,y,z))

In [56]:
len(packets_ev)

83

In [57]:
fig = go.Figure()

fig.add_traces(draw_cathode_planes(
    geom_dict,
    tpc_centers,
    showscale=False,
    opacity=0.3,
    colorscale='Greys',
))

fig.add_traces(draw_anode_planes(
    geom_dict,
    tpc_centers,
    showscale=False,
    opacity=0.1,
    colorscale='ice',
))

for xevent, yevent, zevent in xyzevents:
    trace = go.Scatter3d(
        x=xevent, y=yevent, z=zevent,
        marker_color=dQ,
        name='larpix packets',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=1.
        )
    fig.add_traces(trace)

tracks = plot_tracks(seg_ev, 
                     mode="lines",
                     name="edep segments",
                     visible='legendonly',
                     showlegend=True
                     )

fig.update_layout(
    width=1024, height=768,
    legend_orientation="h",
)

#z scale (0,30)
fig.add_traces(tracks)
fig.show()
#Make sure to click on the "Larpix packets" and "edep segments" icons on the plot so that they are visible!